### surprise 모듈
- 추천시스템 관련 별도의 모듈이 존재한다.
- 아직 자유롭게 쓸만큼 안정화가 안된 불안정?한 모듈이라 버전을 눈여겨 봐야한다
- surprise 모듈의 장점
    - 다양한 추천 알고리즘 탑재
    - surprise는 skelarn 과 API의 명칭과 속성이 아주 유사함
- 잠재요인의 의미
    - user의 특성에 대해 추론하는 요인들을 의미한다. 이 요인들은 여러개의 값들을 축소한 것으로, surprise 모듈에서 사용하는 축소법은 pca가 아니라 svd(sigular value decomposition) 이다. 이것은 gradient decendant 를 이용한 방법이다. 만약 user 3명이라면 user의 잠재요인테이블은 3*1 item은 1*3  이 둘의 연산은 3*3 이 되기 때문에 원래 비어있던 부분을 채워 줄 수 있다.
    - GPU 의 주사용되는 곳 : 게임 랜더링. 주로 단순연산에 강하다. 그러나 GPU의 연산은 내가 알아서 선택할 수 없다. CUDA (텐서플로우 유저들이 사용가능) 는 그것을 가능하게 뚫어놨음. 그래서 cuda 유저들은 gpu 연산을 원하는 대로 선택 가능하다.그래서 흔히들 말하는 gpu는 MBDI의 GPX card를 의미한다. 

In [3]:
import surprise
print(surprise.__version__)

1.1.0


In [4]:
### 실습모듈 
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [5]:
#10만개의 평점 데이터를 무비렌즈에서 불러올 수 있다
data=Dataset.load_builtin('ml-100k')
trainset,testset=train_test_split(data,test_size=25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\admin/.surprise_data/ml-100k


In [6]:
algo=SVD()
algo.fit(trainset)

In [7]:
predictions =algo.test(testset)
print('prediction type : ',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type :  <class 'list'>  size: 25
prediction 결과의 최도 5개 추출


[Prediction(uid='917', iid='50', r_ui=3.0, est=4.027215790648904, details={'was_impossible': False}),
 Prediction(uid='235', iid='318', r_ui=5.0, est=4.336752013275218, details={'was_impossible': False}),
 Prediction(uid='339', iid='12', r_ui=5.0, est=4.893313483809307, details={'was_impossible': False}),
 Prediction(uid='878', iid='755', r_ui=2.0, est=2.86949326312719, details={'was_impossible': False}),
 Prediction(uid='618', iid='56', r_ui=4.0, est=3.141472676196861, details={'was_impossible': False})]

In [8]:
#uid,iid,est 추출
#uid : 유저아이디, iid:아이템아이디(여기서는 영화아이디), r_ui:실제평점, est:예측평점
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('917', '50', 4.027215790648904),
 ('235', '318', 4.336752013275218),
 ('339', '12', 4.893313483809307)]

In [9]:
#predict 매서드는 개별 사용자와 개별 영화에 대한 추천 평점을 반환
#test 매서드는 다수의 사용자에 대해 평점 반환
#두개의 매서드가 분리되어 있음을 주의하라
uid=str(196) #문자열로 입력해야함!
iid=str(382) #문자열로 입력해야함! 
pred=algo.predict(uid,iid)
print(pred)

user: 196        item: 382        r_ui = None   est = 3.67   {'was_impossible': False}


In [10]:
#평가 결과 반환
accuracy.rmse(predictions)

RMSE: 0.9895


0.9894621757866391

#### 실습2
- csv 파일에 바로 접근하기
- surprise는 파일을 읽을 때 컬럼명(a.k.a 헤더)가 있으면 안됌

In [11]:
#순서는 userID, moiveID, rating, timestamp 이다. 
ratings=pd.read_csv('./ml-latest-small/ratings.csv')
ratings.to_csv('./ml-latest-small/ratings_noh.csv', index=False, header=False)

In [12]:
#surprise 에서 바로 읽기
#반드시 Reder에 들어가는 argument의 순서가 user,item,rating 순서대로 들어가야 한다.
#rating_scale : rating 의 최소,최대값 정하기
from surprise import Reader
reader=Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data=Dataset.load_from_file('./ml-latest-small/ratings_noh.csv',reader=reader)

In [13]:
#학습시작
#n_factor : 잠재 요인의 크기
trainset,testset=train_test_split(data,test_size=25, random_state=0)
algo=SVD(n_factors=50, random_state=13)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8305


0.8304727951078185

In [17]:
#pandas 의 데이터를 바로 읽을 수도 있다.
from surprise import Dataset
ratings=pd.read_csv('./ml-latest-small/ratings.csv')
reader=Reader(rating_scale=(0.5,5.0))
data=Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
trainset,testset=train_test_split(data,test_size=25, random_state=0)

algo=SVD(n_factors=50, random_state=13)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8305


0.8304727951078185

In [18]:
#surprise 모듈도 crossvalidation을 지원한다
from surprise.model_selection import cross_validate
ratings=pd.read_csv('./ml-latest-small/ratings.csv')
reader=Reader(rating_scale=(0.5,5.0))
data=Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
algo=SVD(random_state=13)
cross_validate(algo, data, measures=['RMSE','MAE'],cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8726  0.8679  0.8840  0.8725  0.8723  0.8739  0.0054  
MAE (testset)     0.6698  0.6688  0.6781  0.6717  0.6700  0.6717  0.0034  
Fit time          9.31    9.48    8.41    9.51    8.20    8.98    0.56    
Test time         0.17    0.17    0.17    0.41    0.18    0.22    0.10    


{'test_rmse': array([0.87263572, 0.86787814, 0.88400596, 0.8725414 , 0.87228994]),
 'test_mae': array([0.66978772, 0.66876862, 0.6781278 , 0.6717266 , 0.67003626]),
 'fit_time': (9.31110954284668,
  9.478705883026123,
  8.406526565551758,
  9.512566804885864,
  8.20406460762024),
 'test_time': (0.17250394821166992,
  0.16654443740844727,
  0.17157530784606934,
  0.41489171981811523,
  0.18450927734375)}

In [19]:
#GridSearchCV
#n_epochs : 의미있는 잠재요인들을 찾아내는 연산을 몇번할지 정해주는 argument 
from surprise.model_selection import GridSearchCV
param_grid={'n_epochs':[20,40,60], 'n_factors':[50,100,200]}
gs=GridSearchCV(SVD,param_grid,measures=['rmse','mae'],cv=3)
gs.fit(data)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8770504577283244
{'n_epochs': 20, 'n_factors': 50}
